Siemanko
Hejka!!

1.

In [3]:
import tensorflow as tf
(x_train_f, y_train_f), (x_test_f, y_test_f) = tf.keras.datasets.fashion_mnist.load_data()

In [12]:
x_train = x_train_f.reshape((x_train_f.shape[0],-1))
x_test = x_test_f.reshape((x_test_f.shape[0],-1))
y_train = y_train_f.reshape((y_train_f.shape[0],))
y_test = y_test_f

2.

In [5]:
from sklearn.metrics import (accuracy_score, f1_score)
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.svm import SVC

In [6]:
scaler = StandardScaler()

scaler.fit(x_train)

x_train_scaled = scaler.transform(x_train)
x_test_scaled = scaler.transform(x_test)

pca = PCA(n_components=2)
pca.fit(x_train_scaled)

x_train_pca = pca.transform(x_train_scaled)
x_test_pca = pca.transform(x_test_scaled)

In [7]:
clf = SVC(probability=True)
clf.fit(x_train_pca, y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=True, random_state=None, shrinking=True, tol=0.001,
    verbose=False)

In [9]:
y_pred = clf.predict(x_test_pca)

In [10]:
pred = clf.predict_proba(x_test_pca)

In [13]:
accuracy = accuracy_score(y_test, y_pred)
f_score = f1_score(y_test, y_pred, average='weighted')

In [14]:
print(accuracy)
print(f_score)

0.5478
0.5401447577572802


In [2]:
from sklearn.utils import resample
#x_train, y_train = resample(x_train, y_train, n_samples=1500, replace=False, random_state=0)
#x_test, y_test = resample(x_test, y_test, n_samples=200, replace=False, random_state=0)

3a.

In [ ]:
clf_7iter = SVC(probability=True,max_iter=7)

In [ ]:
clf_7iter.fit(x_train, y_train)
y_pred_7ter = clf_7iter.predict(x_test)
pred_7iter = clf_7iter.predict_proba(x_test)

In [ ]:
pred_7iter

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_7iter = accuracy_score(y_test, y_pred_7ter)
accuracy = accuracy_score(y_test, y_pred)

In [ ]:
accuracy_7iter

In [ ]:
accuracy